In [76]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import re
from os import path
from glob import glob
from tqdm import tqdm
import numpy as np
from matplotlib import cm


In [77]:
ROOT_PATH = '\\\\CILab_NAS_Data/homes/inchang/PuzzlePCG/ProxyPlayer'

In [78]:
files = glob(path.join(ROOT_PATH, '*', 'MatchResult_*.csv'))
files

['\\\\CILab_NAS_Data/homes/inchang/PuzzlePCG/ProxyPlayer\\method_mcts_player_0_objective_score_simulation_500\\MatchResult_method_mcts_player_0_objective_score_simulation_500_264c4b52-f819-41f3-b845-769d75890a50.csv',
 '\\\\CILab_NAS_Data/homes/inchang/PuzzlePCG/ProxyPlayer\\method_mcts_player_0_objective_score_simulation_500\\MatchResult_method_mcts_player_0_objective_score_simulation_500_8c1d7068-9881-46ce-b67f-9b3a712bfa4b.csv',
 '\\\\CILab_NAS_Data/homes/inchang/PuzzlePCG/ProxyPlayer\\method_random_player_0\\MatchResult_method_random_player_0_76153839-f9f2-460b-9d84-f0282065e924.csv',
 '\\\\CILab_NAS_Data/homes/inchang/PuzzlePCG/ProxyPlayer\\method_random_player_0\\MatchResult_method_random_player_0_f50f015e-9f58-47b9-a330-8aafa82120a0.csv']

In [79]:
StatisticsColumns = ['Matched_HorizontalPiece', 'Matched_VerticalPiece', 'Matched_CrossPiece', 'Matched_RocketPiece', 'Matched_BombPiece', 'Matched_RainbowPiece']

In [80]:
def parse_filename(data):
    result = dict()
    for i in range(0, len(data), 2):
        result[data[i]] = data[i+1]
    return result

In [103]:
dfs = []
for file_path in tqdm(files):
    # print(file_path)
    file_name = path.basename(file_path)

    _file_name = file_name.split('_')[1:-1]
    args = parse_filename(_file_name)

    df = pd.read_csv(file_path)

    for key, value in args.items():
        df[key] = value
    dfs.append(df)

original_df = pd.concat(dfs)

original_df['Sum_Matched'] = original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece']  + \
    original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece'] + original_df['Matched_HorizontalPiece']
original_df.loc[original_df['method'] == 'random', 'simulation'] = 0

original_df

100%|██████████| 4/4 [00:00<00:00, 190.43it/s]


,EpisodeCount,StepCount,Time,InstanceUUID,SettleCount,ChangedCount,MeanComparisonCount,StdComparisonCount,Matched_HorizontalPiece,Matched_VerticalPiece,...,Target_VerticalPiece,Target_CrossPiece,Target_RocketPiece,Target_BombPiece,Target_RainbowPiece,method,player,objective,simulation,Sum_Matched
0,1,300,2023-05-14 11:28:22.575,264c4b52-f819-41f3-b845-769d75890a50,0,102,18.39333,71.38317,44,37,...,3,0,2,2,3,mcts,0,score,500,264
1,2,300,2023-05-14 11:29:22.519,264c4b52-f819-41f3-b845-769d75890a50,0,90,18.29667,73.23780,56,37,...,3,0,2,2,3,mcts,0,score,500,336
2,3,301,2023-05-14 11:30:22.419,264c4b52-f819-41f3-b845-769d75890a50,0,89,15.30565,63.59476,102,43,...,3,0,2,2,3,mcts,0,score,500,612
3,4,302,2023-05-14 11:31:21.785,264c4b52-f819-41f3-b845-769d75890a50,0,95,13.88742,60.08361,70,24,...,3,0,2,2,3,mcts,0,score,500,420
4,5,301,2023-05-14 11:32:20.636,264c4b52-f819-41f3-b845-769d75890a50,0,81,17.11628,72.40711,46,32,...,3,0,2,2,3,mcts,0,score,500,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,300,2023-05-14 14:10:41.623,f50f015e-9f58-47b9-a330-8aafa82120a0,0,0,0.00000,0.00000,44,47,...,3,0,2,2,3,random,0,NaN,0,264
996,997,303,2023-05-14 14:10:51.083,f50f015e-9f58-47b9-a330-8aafa82120a0,0,0,0.00000,0.00000,57,47,...,3,0,2,2,3,random,0,NaN,0,342
997,998,300,2023-05-14 14:11:01.196,f50f015e-9f58-47b9-a330-8aafa82120a0,0,0,0.00000,0.00000,53,34,...,3,0,2,2,3,random,0,NaN,0,318
998,999,302,2023-05-14 14:11:10.520,f50f015e-9f58-47b9-a330-8aafa82120a0,0,0,0.00000,0.00000,40,37,...,3,0,2,2,3,random,0,NaN,0,240


In [104]:
error_result = original_df.groupby(['method', 'simulation'])['Matched_HorizontalPiece',
                                      'Matched_VerticalPiece',
                                      'Matched_CrossPiece',
                                      'Matched_RocketPiece',
                                      'Matched_BombPiece',
                                      'Matched_RainbowPiece',
                                      'Sum_Matched'].agg(
    {
        'Matched_HorizontalPiece':['mean','std'],
        'Matched_VerticalPiece':['mean','std'],
        'Matched_CrossPiece':['mean','std'],
        'Matched_RocketPiece':['mean','std'],
        'Matched_BombPiece':['mean','std'],
        'Matched_RainbowPiece':['mean','std'],
        'Sum_Matched':['mean','std'],
     }
)
error_result

C:\Users\GIST\.conda\envs\mlagents\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


Matched_HorizontalPiece            Matched_VerticalPiece  \
                                     mean        std                  mean   
method simulation                                                            
mcts   500                      54.450635  12.515200             36.264457   
random 0                        46.447000  10.365564             32.832000   

                            Matched_CrossPiece           Matched_RocketPiece  \
                        std               mean       std                mean   
method simulation                                                              
mcts   500         9.063558           7.288434  4.562050           93.078984   
random 0           7.955088           6.202000  3.947529           83.756000   

                             Matched_BombPiece           Matched_RainbowPiece  \
                         std              mean       std                 mean   
method simulation                                                               
mcts   500         18.833392          7.311707  4.382894             9.576869   
random 0           15.879312          6.128000  3.853445             8.096000   

                            Sum_Matched             
                        std        mean        std  
method simulation                                   
mcts   500         5.346572  326.703808  75.091201  
random 0           4.611444  278.682000  62.193383